In [1]:
import requests
import json
import random
import time
import pickle


model = "dRIT_pi"

template = {
  "Answer": "",
  "Score Lexical Semantics": "",
  "Score Predicate-Argument Structure": "",
  "Score Logic": "",
  "Score Knowledge & Common Sense": ""
}


# TRAIN de Scitail

In [2]:
import pandas as pd

In [4]:
df =pd.read_pickle("../predictions/RL_Scitail_DEV_600.pickle")
df

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
0,2,0.400000,0.0,0.600000,0.594089,0.140100,4.340529,0.762486,0.666667,0,0,0.400000,0,0,0.600000,entails,Below the earth's crust is a layer known as th...,The mantle layer of earth is located just belo...
1,2,0.375000,0.0,0.166667,0.605406,0.171515,4.381443,0.918428,0.875000,1,0,0.708586,0,0,0.625000,entails,Meiosis is the two-stage process of cell divis...,"Gametogenesis, the production of sperm and egg..."
2,2,0.750000,0.0,0.333333,0.571548,0.151636,4.435317,0.862003,0.857143,1,0,0.586111,0,0,0.250000,entails,The central nervous system acts like a telepho...,The nervous system sends electrical signals to...
3,2,0.333333,0.0,0.666667,0.447214,0.152032,4.516060,0.882395,0.800000,0,0,0.429167,0,0,0.666667,entails,Chemists often use as a unit of pressure the t...,A torr is a unit used to measure pressure.
4,2,0.400000,0.0,0.750000,0.707107,0.055214,4.403386,0.889384,0.800000,0,0,0.775000,0,0,0.600000,entails,The liver makes urea and the kidney puts the u...,Urea is made in the liver and excreted in urine.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,2,0.400000,0.0,0.250000,0.617213,0.178026,4.306672,0.622818,0.600000,0,0,0.456349,0,0,0.600000,neutral,Flagella and cilia are extensions of the plasm...,The plasma membrane controls what goes in and ...
596,2,0.625000,0.0,0.333333,0.354787,0.010448,4.223442,0.669017,0.727273,3,0,0.607912,0,0,0.375000,neutral,Introduction People have wondered for a long t...,This process usually takes the longest amount ...
597,2,0.666667,0.0,0.000000,0.471405,0.189161,3.957897,0.353976,0.375000,1,0,0.370370,0,0,0.333333,neutral,Infancy and Childhood,An individual grows quickly and develops new a...
598,2,0.500000,0.0,0.400000,0.670820,0.165602,4.410536,0.775091,0.833333,1,0,0.716667,0,0,0.500000,neutral,Subject matter wherein the light is randomly s...,Translucent matter transmits light but scatter...


In [5]:
lista_respuestasOllama=[]

In [6]:
df.columns

Index(['relation', 'no_matcheadas', 'contradiction', 'jaccard', 'simBoW',
       'sumas', 'mutinf_t', 'max_info_t', 'entail', 'list_comp', 'list_incomp',
       'Jaro-Winkler_rit', 'negT', 'negH', 'overlap_ent', 'clases',
       'sentence_A', 'sentence_B'],
      dtype='object')

In [7]:
faltantes=[]

In [8]:
df_t=df.copy()
df_t

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
0,2,0.400000,0.0,0.600000,0.594089,0.140100,4.340529,0.762486,0.666667,0,0,0.400000,0,0,0.600000,entails,Below the earth's crust is a layer known as th...,The mantle layer of earth is located just belo...
1,2,0.375000,0.0,0.166667,0.605406,0.171515,4.381443,0.918428,0.875000,1,0,0.708586,0,0,0.625000,entails,Meiosis is the two-stage process of cell divis...,"Gametogenesis, the production of sperm and egg..."
2,2,0.750000,0.0,0.333333,0.571548,0.151636,4.435317,0.862003,0.857143,1,0,0.586111,0,0,0.250000,entails,The central nervous system acts like a telepho...,The nervous system sends electrical signals to...
3,2,0.333333,0.0,0.666667,0.447214,0.152032,4.516060,0.882395,0.800000,0,0,0.429167,0,0,0.666667,entails,Chemists often use as a unit of pressure the t...,A torr is a unit used to measure pressure.
4,2,0.400000,0.0,0.750000,0.707107,0.055214,4.403386,0.889384,0.800000,0,0,0.775000,0,0,0.600000,entails,The liver makes urea and the kidney puts the u...,Urea is made in the liver and excreted in urine.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,2,0.400000,0.0,0.250000,0.617213,0.178026,4.306672,0.622818,0.600000,0,0,0.456349,0,0,0.600000,neutral,Flagella and cilia are extensions of the plasm...,The plasma membrane controls what goes in and ...
596,2,0.625000,0.0,0.333333,0.354787,0.010448,4.223442,0.669017,0.727273,3,0,0.607912,0,0,0.375000,neutral,Introduction People have wondered for a long t...,This process usually takes the longest amount ...
597,2,0.666667,0.0,0.000000,0.471405,0.189161,3.957897,0.353976,0.375000,1,0,0.370370,0,0,0.333333,neutral,Infancy and Childhood,An individual grows quickly and develops new a...
598,2,0.500000,0.0,0.400000,0.670820,0.165602,4.410536,0.775091,0.833333,1,0,0.716667,0,0,0.500000,neutral,Subject matter wherein the light is randomly s...,Translucent matter transmits light but scatter...


# RIT base

In [9]:
for i in range(1):
    for index,strings in df_t[i*600:(i+1)*600].iterrows():
        prompt = ''''Analyze the following sentences: 
        Text: '''+strings["sentence_A"]+'''
        Hypothesis: '''+ strings["sentence_B"]+'''
        plus next features:
        sums:'''+ str(strings["sumas"])+'''
        negH:'''+ str(strings["negH"])+'''
        list_compability:'''+ str(strings["list_comp"])+'''
        list_incompability:'''+ str(strings["list_incomp"])+'''
        relation:'''+ str(strings["relation"])+'''
        entail:'''+ str(strings["entail"])+'''
        negT:'''+ str(strings["negT"])+'''
        mutinf:'''+ str(strings["mutinf_t"])+'''
        contradiction:'''+ str(strings["contradiction"])+'''
        no_matcheadas:'''+ str(strings["no_matcheadas"])+'''
        Jaro-Winkler_rit:'''+ str(strings["Jaro-Winkler_rit"])+'''
        simBoW:'''+ str(strings["simBoW"])+'''
        jaccard:'''+ str(strings["jaccard"])+'''
        overlap_ent:'''+ str(strings["overlap_ent"])+'''
        max_info_t:'''+ str(strings["max_info_t"])+''' 
        only responds using the template. '''
        #print(prompt)
        data = {
            "prompt": prompt,
            "model": model,
            "format": "json",
            "stream": False,
            #"options": {"temperature": 0.2},
        }
        try:
            response = requests.post("http://localhost:11434/api/generate", json=data, stream=False,timeout=15)        
            json_data = json.loads(response.text)
            lista_respuestasOllama.append(json.dumps(json.loads(json_data["response"]), indent=2))
            print(index)
        except:
            print("Saltó",index)
            faltantes.append(index)
            lista_respuestasOllama.append("NA")
    with open("resultados/rit_pi_Scitail_DEV_600_"+str(i+1)+".pickle", "wb") as f:
        pickle.dump(lista_respuestasOllama, f)
    print("Primer rango: ",(i)*600,(i+1)*600)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [10]:
print(len(lista_respuestasOllama))

600
